### To test correlations between HEXACO lexical analysis scores and HEXACO-PI-R 100 scores.

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np

scores_file = 'data/popc_results.csv'
scores_data = pd.read_csv(scores_file, index_col=0)
to_fix = [x for x in scores_data.index if "'" in x]
print(f'Fixing names: {", ".join(to_fix)}.')
for n in to_fix:
    scores_data.rename(index={n:n.replace("'", '')}, inplace=True)
print(f"Read in {len(scores_data)} responses from {scores_file}")

from support.data_support import load_agent_data
agent_loadings = load_agent_data('intermediate/loadings_popc_10.csv', None)
agent_loadings.columns = ['Dishonesty', 
                          'Disagreeableness', 
                          'Introversion', 
                          'Unconscientiousness', 
                          'Unheroic', 
                          'Unscholarly', 
                          'Gendered-emotionality', 
                          'Unsentimentality', 
                          'Insensitivity', 
                          'Unartistic']

Fixing names: PatrickO'Connor, Margaret'Maggie'Dupont, MartinO'Donnell, Johnathan'BigJohn'Moretti, JohnO'Reilly.
Read in 310 responses from data/popc_results.csv


In [2]:
# calculate score for the agents, based on the recovered factors' loadings.
df_dimensions = pd.DataFrame(index=scores_data.index)
for factor in agent_loadings.columns:
    terms = list(agent_loadings[factor].dropna().index)
    weights = agent_loadings[factor].dropna()
    scores = [x-5 for x in scores_data[terms].values]
    scores = np.array(scores)
    scores = scores * weights.values
    df_dimensions[factor] = scores.sum(axis=1)
    
df_dimensions

,Dishonesty,Disagreeableness,Introversion,Unconscientiousness,Unheroic,Unscholarly,Gendered-emotionality,Unsentimentality,Insensitivity,Unartistic
DaveThomas,-62.461978,32.689884,-22.844029,0.629958,-7.230852,4.994465,-20.026061,-9.318992,16.388300,-10.486598
OliverHarris,-62.342512,30.242595,-12.104332,1.857714,-0.961591,15.342589,-16.715384,7.987371,19.775263,-0.712701
DavidHarrison,-62.324809,19.482030,-20.569922,0.136957,-14.749656,1.268159,-16.992963,-9.809272,0.419046,11.733863
JerryMcAlister,36.619510,-4.315614,-55.004941,-29.695646,12.753347,2.039355,-14.259798,-18.774245,-17.063799,1.993753
GregoryHarrison,16.567249,2.801788,28.037754,-0.807457,-6.542806,2.103973,-29.922356,-15.288581,4.941060,-5.039512
...,...,...,...,...,...,...,...,...,...,...
EmmaJohnson,-66.467217,-44.141327,-51.191096,-31.844623,-3.542617,-9.703306,-11.829826,-5.926566,-13.717788,-13.233424
AmeliaMartinez,-66.397745,-46.537180,-64.119699,-18.675703,3.778099,-5.979286,-4.144421,-9.018421,-16.601365,-7.951785
CarmenSanchez,-67.182226,-52.987126,-58.240059,-0.359607,-13.361652,-3.828874,-9.097542,3.721637,8.778540,-6.890492
LiamJohnson,-54.787110,-43.205908,-39.230955,-16.890873,5.851213,-15.283781,-16.961778,-17.853390,-31.238783,-26.373939


In [3]:
# now load in the scores from the HEXACO-PI-R 100 survey.
from support.hexaco_pi_r import responses_to_scores, scoring, calc_dimension_scores
pir100 = pd.read_csv('data/hexaco-pi-r-responses-GPT-4.csv', index_col=0)
print(f"Read in {len(pir100)} sets of survey responses (questions={pir100.shape[1]}).")
pir100.rename(index={'CalebJThompson':'CalebJ'}, inplace=True)
pir100.rename(index={'JohnathonKRiley':'JohnathonK'}, inplace=True)
pir100.rename(index={'VioletBRichmond':'VioletB'}, inplace=True)
pir100.columns = range(len(pir100.columns))

# map text responses to score, using scoring mechanism (includes reverse scoring)
pir100_scores = responses_to_scores(pir100, scoring)

# then figure out a score for each dimension for each agent.
pir100_dim_scores = calc_dimension_scores(pir100_scores, scoring)
pir100_dim_scores

Read in 310 sets of survey responses (questions=100).


,Honesty-Humility,Emotionality,Extraversion,Agreeableness,Conscientiousness,Openness
DaveThomas,4.125,2.5625,3.125,2.5625,4.9375,3.0
OliverHarris,4.375,3.25,3.75,2.5,4.875,3.6875
DavidHarrison,4.0625,2.0625,3.75,3.125,4.9375,3.375
JerryMcAlister,2.75,2.75,4.3125,2.625,4.5,3.0
GregoryHarrison,3.5625,2.375,3.125,2.625,4.75,3.3125
...,...,...,...,...,...,...
EmmaJohnson,4.0,3.5625,3.625,3.5,3.5,3.5625
AmeliaMartinez,4.375,3.6875,4.125,3.5625,3.875,3.9375
CarmenSanchez,4.25,3.9375,3.5625,3.5,4.625,3.3125
LiamJohnson,4.0,2.9375,3.75,3.375,2.3125,3.375


In [4]:
from scipy.stats import pearsonr

# now select the best 6 factors (semantic similarity with hexaco)
# to match the 6 hexaco factors.
selection_df = df_dimensions.copy()
selection_df = selection_df[['Dishonesty', 'Gendered-emotionality', 
                             'Introversion', 'Disagreeableness', 
                             'Unconscientiousness', 'Unscholarly']]
selection_df.columns = ['Honesty-Humility', 'Emotionality', 
                        'Extraversion', 'Agreeableness', 
                        'Conscientiousness', 'Openness']

results_df = pd.DataFrame(index=selection_df.columns)
results = {}
for dim in selection_df.columns:
    r, p_value = pearsonr(selection_df[dim], pir100_dim_scores[dim])
    results_df.loc[dim, 'r'] = r
    results_df.loc[dim, 'p_value'] = p_value
    print(f"{dim}: r={r:.3f}, p={p_value:.3f}")
    results[dim] = pearsonr(selection_df[dim], pir100_dim_scores[dim])

pp = pd.DataFrame(results).T
pp = pp.round(4)
pp.columns = ['r', 'p']
pp

Honesty-Humility: r=-0.755, p=0.000
Emotionality: r=0.467, p=0.000
Extraversion: r=-0.780, p=0.000
Agreeableness: r=-0.824, p=0.000
Conscientiousness: r=0.453, p=0.000
Openness: r=0.273, p=0.000


,r,p
Honesty-Humility,-0.7547,0.0
Emotionality,0.4665,0.0
Extraversion,-0.7800,0.0
Agreeableness,-0.8244,0.0
Conscientiousness,0.4535,0.0
Openness,0.2725,0.0


In [5]:
# now for all hexaco dimensions, across all agent factors, report pearsons!   
results = pd.DataFrame(index=pir100_dim_scores.columns, columns=df_dimensions.columns)
for hexaco_survey in pir100_dim_scores.columns:
    for agents_factor in df_dimensions.columns:
        r, p_value = pearsonr(pir100_dim_scores[hexaco_survey], df_dimensions[agents_factor])
        results.loc[hexaco_survey, agents_factor] = r

pp = pd.DataFrame(results)
pp = pp.round(4)
pp

,Dishonesty,Disagreeableness,Introversion,Unconscientiousness,Unheroic,Unscholarly,Gendered-emotionality,Unsentimentality,Insensitivity,Unartistic
Honesty-Humility,-0.75471,-0.133522,0.175344,0.165627,-0.283419,-0.04419,0.044383,0.403054,0.114002,-0.505696
Emotionality,-0.306589,-0.466785,-0.517131,0.054199,-0.152814,-0.193045,0.466502,0.366426,0.171477,0.112903
Extraversion,0.100355,-0.095134,-0.779981,-0.421186,0.459744,-0.181832,0.49837,-0.087461,-0.359913,0.246481
Agreeableness,-0.134399,-0.824396,-0.26371,-0.294004,-0.124907,-0.065382,0.370708,0.361597,-0.405527,-0.111624
Conscientiousness,0.119597,0.429748,0.3036,0.453481,-0.318127,0.269202,-0.316112,-0.049174,0.559718,0.300105
Openness,0.034203,-0.252212,-0.066494,0.05375,0.084595,0.272539,0.193962,0.055992,0.069407,0.372826


### Now same for Llama 3.2's responses

In [6]:
# now load in the scores from the HEXACO-PI-R 100 survey.
from support.hexaco_pi_r import responses_to_scores, scoring, calc_dimension_scores
pir100 = pd.read_csv('data/hexaco-pi-r-responses-llama3.2.csv', index_col=0)
print(f"Read in {len(pir100)} sets of survey responses (questions={pir100.shape[1]}).")
pir100.rename(index={'CalebJThompson':'CalebJ'}, inplace=True)
pir100.rename(index={'JohnathonKRiley':'JohnathonK'}, inplace=True)
pir100.rename(index={'VioletBRichmond':'VioletB'}, inplace=True)
pir100.columns = range(len(pir100.columns))

# map text responses to score, using scoring mechanism (includes reverse scoring)
pir100_scores = responses_to_scores(pir100, scoring)

# then figure out a score for each dimension for each agent.
pir100_dim_scores = calc_dimension_scores(pir100_scores, scoring)
pir100_dim_scores

Read in 310 sets of survey responses (questions=100).


,Honesty-Humility,Emotionality,Extraversion,Agreeableness,Conscientiousness,Openness
DaveThomas,3.5625,2.875,2.625,2.9375,3.4375,3.4375
OliverHarris,3.5625,3.25,2.6875,3.1875,3.25,2.875
DavidHarrison,3.5625,2.9375,2.9375,3.1875,3.25,3.125
JerryMcAlister,3.5625,3.0,2.9375,2.9375,2.875,3.0625
GregoryHarrison,3.5625,3.0,3.3125,2.6875,3.1875,3.375
...,...,...,...,...,...,...
EmmaJohnson,3.3125,3.375,2.8125,3.125,2.875,3.25
AmeliaMartinez,3.9375,2.9375,3.0,2.8125,3.1875,3.25
CarmenSanchez,3.6875,3.125,3.3125,3.125,3.25,3.125
LiamJohnson,3.25,3.3125,2.9375,2.75,2.875,3.125


In [7]:
from scipy.stats import pearsonr

# now select the best 6 factors (semantic similarity with hexaco)
# to match the 6 hexaco factors.
selection_df = df_dimensions.copy()
selection_df = selection_df[['Dishonesty', 'Gendered-emotionality', 
                             'Introversion', 'Disagreeableness', 
                             'Unconscientiousness', 'Unscholarly']]
selection_df.columns = ['Honesty-Humility', 'Emotionality', 
                        'Extraversion', 'Agreeableness', 
                        'Conscientiousness', 'Openness']

results_df = pd.DataFrame(index=selection_df.columns)
results = {}
for dim in selection_df.columns:
    r, p_value = pearsonr(selection_df[dim], pir100_dim_scores[dim])
    results_df.loc[dim, 'r'] = r
    results_df.loc[dim, 'p_value'] = p_value
    print(f"{dim}: r={r:.3f}, p={p_value:.3f}")
    results[dim] = pearsonr(selection_df[dim], pir100_dim_scores[dim])

pp = pd.DataFrame(results).T
pp = pp.round(4)
pp.columns = ['r', 'p']
pp

Honesty-Humility: r=-0.190, p=0.001
Emotionality: r=0.098, p=0.083
Extraversion: r=-0.219, p=0.000
Agreeableness: r=-0.315, p=0.000
Conscientiousness: r=0.171, p=0.003
Openness: r=-0.042, p=0.463


,r,p
Honesty-Humility,-0.1895,0.0008
Emotionality,0.0985,0.0834
Extraversion,-0.2192,0.0001
Agreeableness,-0.3146,0.0000
Conscientiousness,0.1708,0.0025
Openness,-0.0419,0.4627


In [8]:
# now for all hexaco dimensions, across all agent factors, report pearsons!   
results = pd.DataFrame(index=pir100_dim_scores.columns, columns=df_dimensions.columns)
for hexaco_survey in pir100_dim_scores.columns:
    for agents_factor in df_dimensions.columns:
        r, p_value = pearsonr(pir100_dim_scores[hexaco_survey], df_dimensions[agents_factor])
        results.loc[hexaco_survey, agents_factor] = r

pp = pd.DataFrame(results)
pp = pp.round(4)
pp

,Dishonesty,Disagreeableness,Introversion,Unconscientiousness,Unheroic,Unscholarly,Gendered-emotionality,Unsentimentality,Insensitivity,Unartistic
Honesty-Humility,-0.189521,-0.120744,0.002962,-0.099523,-0.068352,-0.083713,0.002017,0.061063,-0.113472,-0.264907
Emotionality,-0.056281,-0.158783,-0.14786,-0.150794,0.128592,-0.138276,0.098479,-0.006242,-0.097574,-0.074996
Extraversion,-0.030911,-0.046236,-0.219151,-0.148663,0.055894,-0.100407,0.079303,-0.024444,-0.149564,0.04081
Agreeableness,-0.087454,-0.31458,-0.151423,-0.142468,-0.078533,-0.026979,0.202009,0.180279,-0.175593,-0.022319
Conscientiousness,-0.087988,0.169219,0.185521,0.170843,-0.06027,-0.01243,-0.255381,-0.201689,0.19704,0.058344
Openness,-0.218463,-0.099445,-0.048667,-0.036089,-0.051234,-0.041865,0.010441,-0.026436,-0.029505,-0.019232
